Pada tugas kali ini, kita akan mencoba untuk membuat model dari data training untuk mengklasifikasikan data_uji.

Kita ingin melihat dengan fitur yang dimiliki apakah penumpang tersebut akan selamat atau tidak.

Algoritma yang akan kita gunakan adalah logistic regression.


In [ ]:
#import semua package/library pada sel ini, jika butuh memasukkan package/library baru maka run ulang sel ini

from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import sklearn.feature_selection as fs
from sklearn import metrics

2. Data Collection


In [ ]:
#panggil data kalian dan masukkan pada dua variabel data_mentah untuk train dan data_uji untuk data_uji

data_mentah = pd.read_csv("train.csv")
data_uji = pd.read_csv("data_uji.csv")

In [ ]:
#tampilkan head dari data_mentah
display(data_mentah.head())

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,Family_Size
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr,1
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Mrs,1
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Miss,0
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Mrs,1
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Mr,0


In [ ]:
#tampilkan head dari data_uji
display(data_uji.head())

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,Family_Size
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911,Mr,0
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272,Mrs,1
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276,Mr,0
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154,Mr,0
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298,Mrs,2


3. Data Preparation

Pada bagian ini kita akan memperbaiki nan value

In [ ]:
def check_values(df):
    data = []
    for col in df.columns:
        data.append([col, \
                  df[col].dtype, \
                  df[col].isna().sum(), \
                  round(100*(df[col].isna().sum()/len(df)), 2), \
                  df[col].nunique(), \
                  df[col].unique()
                  ])
    
    return pd.DataFrame(columns=['kolom', 'dataType', 'null', 'nullPct', 'unique', 'values'], data=data)

In [ ]:
#Check detail dan nilai NaN pada data_mentah. gunakan print untuk Nilai NaN
check_values(data_mentah)

,kolom,dataType,null,nullPct,unique,values
0,Age,float64,0,0.0,89,"[22.0, 38.0, 26.0, 35.0, 30.0, 54.0, 2.0, 27.0..."
1,Cabin,object,687,77.1,147,"[nan, C85, C123, E46, G6, C103, D56, A6, C23 C..."
2,Embarked,object,0,0.0,3,"[S, C, Q]"
3,Fare,float64,0,0.0,248,"[7.25, 71.2833, 7.925, 53.1, 8.05, 8.4583, 51...."
4,Name,object,0,0.0,891,"[Braund, Mr. Owen Harris, Cumings, Mrs. John B..."
5,Parch,int64,0,0.0,7,"[0, 1, 2, 5, 3, 4, 6]"
6,PassengerId,int64,0,0.0,891,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
7,Pclass,int64,0,0.0,3,"[3, 1, 2]"
8,Sex,object,0,0.0,2,"[male, female]"
9,SibSp,int64,0,0.0,7,"[1, 0, 3, 4, 2, 5, 8]"


In [ ]:
data_mentah.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Family_Size
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.445196,32.204208,0.381594,446.000000,2.308642,0.523008,0.383838,0.904602
std,13.244896,49.693429,0.806057,257.353842,0.836071,1.102743,0.486592,1.613459
min,0.420000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,22.000000,7.910400,0.000000,223.500000,2.000000,0.000000,0.000000,0.000000
50%,30.000000,14.454200,0.000000,446.000000,3.000000,0.000000,0.000000,0.000000
75%,35.500000,31.000000,0.000000,668.500000,3.000000,1.000000,1.000000,1.000000
max,80.000000,512.329200,6.000000,891.000000,3.000000,8.000000,1.000000,10.000000


In [ ]:
#Check detail dan nilai NaN pada data_uji. gunakan print untuk Nilai NaN
check_values(data_uji)

,kolom,dataType,null,nullPct,unique,values
0,Age,float64,0,0.0,10,"[34.5, 47.0, 62.0, 27.0, 22.0, 14.0, 30.0, 26...."
1,Cabin,float64,10,100.0,0,[nan]
2,Embarked,object,0,0.0,3,"[Q, S, C]"
3,Fare,float64,0,0.0,10,"[7.8292, 7.0, 9.6875, 8.6625, 12.2875, 9.225, ..."
4,Name,object,0,0.0,10,"[Kelly, Mr. James, Wilkes, Mrs. James (Ellen N..."
5,Parch,int64,0,0.0,2,"[0, 1]"
6,PassengerId,int64,0,0.0,10,"[892, 893, 894, 895, 896, 897, 898, 899, 900, ..."
7,Pclass,int64,0,0.0,2,"[3, 2]"
8,Sex,object,0,0.0,2,"[male, female]"
9,SibSp,int64,0,0.0,3,"[0, 1, 2]"


In [ ]:
data_uji.describe()

,Age,Cabin,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Family_Size
count,10.000000,0.0,10.000000,10.000000,10.00000,10.000000,10.000000,0.0,10.000000
mean,30.150000,NaN,12.270010,0.200000,896.50000,2.800000,0.500000,NaN,0.700000
std,14.541225,NaN,7.777627,0.421637,3.02765,0.421637,0.707107,NaN,0.948683
min,14.000000,NaN,7.000000,0.000000,892.00000,2.000000,0.000000,NaN,0.000000
25%,21.250000,NaN,7.679200,0.000000,894.25000,3.000000,0.000000,NaN,0.000000
50%,26.500000,NaN,8.943750,0.000000,896.50000,3.000000,0.000000,NaN,0.000000
75%,33.375000,NaN,11.637500,0.000000,898.75000,3.000000,1.000000,NaN,1.750000
max,62.000000,NaN,29.000000,1.000000,901.00000,3.000000,2.000000,NaN,2.000000


Terlihat untuk data cabin memiliki banyak nilai NaN. Jadi akan kita drop data tersebut. Name, Ticket, Title juga akan kita drop karena memiliki banyak nilai unik dan berbentuk string.

In [ ]:
#drop kolom yang tidak dibutuhkan dari kedua dataframe
data_mentah = data_mentah.drop(columns=['Cabin', 'Name', 'Ticket', 'Title'])
data_uji = data_uji.drop(columns=['Cabin', 'Name', 'Ticket', 'Title'])

In [ ]:
#tampilkan head mentah
data_mentah.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,22.0,S,7.2500,0,1,3,male,1,0.0,1
1,38.0,C,71.2833,0,2,1,female,1,1.0,1
2,26.0,S,7.9250,0,3,3,female,0,1.0,0
3,35.0,S,53.1000,0,4,1,female,1,1.0,1
4,35.0,S,8.0500,0,5,3,male,0,0.0,0


In [ ]:
#tampilkan head uji
data_uji.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,34.5,Q,7.8292,0,892,3,male,0,NaN,0
1,47.0,S,7.0000,0,893,3,female,1,NaN,1
2,62.0,Q,9.6875,0,894,2,male,0,NaN,0
3,27.0,S,8.6625,0,895,3,male,0,NaN,0
4,22.0,S,12.2875,1,896,3,female,1,NaN,2


In [ ]:
#sekarang kita akan mengubah male : 0 dan female menjadi 1 pada kolom Sex. S = 1, C = 2, Q = 3 pada kolom embarked. Ubah kedua dataframe. 
#tampilkan head pada data mentah
#manfaatkan modul replace

data_mentah['Sex'] = data_mentah['Sex'].replace(['male'], '0')
data_mentah['Sex'] = data_mentah['Sex'].replace(['female'], '1')
data_uji['Sex'] = data_uji['Sex'].replace(['male'], '0')
data_uji['Sex'] = data_uji['Sex'].replace(['female'], '1')

In [ ]:
data_mentah.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,22.0,S,7.2500,0,1,3,0,1,0.0,1
1,38.0,C,71.2833,0,2,1,1,1,1.0,1
2,26.0,S,7.9250,0,3,3,1,0,1.0,0
3,35.0,S,53.1000,0,4,1,1,1,1.0,1
4,35.0,S,8.0500,0,5,3,0,0,0.0,0


In [ ]:
data_uji.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,34.5,Q,7.8292,0,892,3,0,0,NaN,0
1,47.0,S,7.0000,0,893,3,1,1,NaN,1
2,62.0,Q,9.6875,0,894,2,0,0,NaN,0
3,27.0,S,8.6625,0,895,3,0,0,NaN,0
4,22.0,S,12.2875,1,896,3,1,1,NaN,2


Sekarang kita akan membagi data_mentah menjadi data training dan test


In [ ]:
#pilih data yang akan menjadi fitur dan label. gunakan data_mentah karena kita akan membuat model
X = data_mentah[['Age', 'Fare', 'Parch', 'PassengerId', 'Pclass', 'Sex', 'SibSp', 'Family_Size']]
y = data_mentah[['Survived']]

In [ ]:
#import sklearn.feature_selection pada sel package/library aliaskan sebagai fs
#import module LogisticRegression dari sklearn.linear_model. taruh pada sel package/library

model_lr = LogisticRegression()
rfe = fs.RFE(model_lr)
rfe.fit(X,y)
print(f'Support = {rfe.support_}')
print(f'Ranking = {rfe.ranking_}')

Support = [False False False False  True  True  True  True]
Ranking = [3 4 2 5 1 1 1 1]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

In [ ]:
#sekarang kita ubah nilai X dengan mengikuti nilai kolom terbaik. urutan pada list ranking sesuai urutan pada saat inisiasi variabel X
# pilih ranking 1

X = data_mentah[['Pclass', 'Sex', 'SibSp', 'Family_Size']]

In [ ]:
'''import/tambahkan semua package/library pada sel paling atas(terkait library)
masukkan kedua kolom tersebut pada variabel yang bersesuaian.
kita bagi menjadi data training dan test dengan test_size = 0.2
'''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
print(f'dimensi dari X_train : {len(X_train)}')
print(f'dimensi dari y_train : {len(y_train)}')
print(f'dimensi dari X_test : {len(X_test)}')
print(f'dimensi dari y_test : {len(y_test)}')

dimensi dari X_train : 712
dimensi dari y_train : 712
dimensi dari X_test : 179
dimensi dari y_test : 179


4. Data Processing

In [ ]:
#training model
#langsung gunakan object logistic regression yang sudah dibuat sebelumnya

# fit the model with data
model_lr.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [ ]:
#Evaluasi
#kita buat variabel y_predict
#import modul metrics dari sklearn. Taruh pada sel package/library di awal
#test nilai AUC

y_predict = model_lr.predict(X_test)
auc = metrics.accuracy_score(y_test, y_predict)
print(f'Nilai AUC : {auc}')

Nilai AUC : 0.8268156424581006


Sekarang kita akan mencoba mengklasifikasikan data_uji kita

In [ ]:
#1. buat sebuah variabel untuk menampung kolom dari data_uji. Ingat kolom yang dipilih harus sama dengan semua kolom pada saat memilih X
#2. lakukan prediksi dengan menggunakan variabel pada poin satu.
#3. masukkan variabel pada poin dua pada data uji kolom Survived
#4. tampilkan data_uji

x = data_uji[['Pclass', 'Sex', 'SibSp', 'Family_Size']]

y_predict = model_lr.predict(x)

TypeError: ignored

In [ ]:
check_values(data_uji)

,kolom,dataType,null,nullPct,unique,values
0,Age,float64,0,0.0,10,"[34.5, 47.0, 62.0, 27.0, 22.0, 14.0, 30.0, 26...."
1,Embarked,object,0,0.0,3,"[Q, S, C]"
2,Fare,float64,0,0.0,10,"[7.8292, 7.0, 9.6875, 8.6625, 12.2875, 9.225, ..."
3,Parch,int64,0,0.0,2,"[0, 1]"
4,PassengerId,int64,0,0.0,10,"[892, 893, 894, 895, 896, 897, 898, 899, 900, ..."
5,Pclass,int64,0,0.0,2,"[3, 2]"
6,Sex,object,0,0.0,2,"[0, 1]"
7,SibSp,int64,0,0.0,3,"[0, 1, 2]"
8,Survived,float64,10,100.0,0,[nan]
9,Family_Size,int64,0,0.0,3,"[0, 1, 2]"


In [ ]:
#tuliskan hasil analisa kalian disini